In [15]:
import torch
from covid_ode import sidartha_ode, covid_dataset
import pandas as pd
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm

# from ode_nn import Dataset_graph, train_epoch_graph, eval_epoch_graph, get_lr
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [16]:
canada = pd.read_csv('canada.csv')

In [17]:
from sklearn.preprocessing import MinMaxScaler

In [18]:
canada.shape

(555, 4)

In [19]:
TRAIN_IDX = int(canada.shape[0]*.5)
VAL_IDX = TRAIN_IDX + int(canada.shape[0]*.1)

In [20]:
TRAIN_IDX, VAL_IDX

(277, 332)

In [21]:
canada

,Confirmed,Recovered,Deaths,Last_Update
0,4.0,0.0,0.0,2020-01-28 23:00:00
1,4.0,0.0,0.0,2020-01-29 19:30:00
2,4.0,0.0,0.0,2020-01-30 16:00:00
3,4.0,0.0,0.0,2020-01-31 23:59:00
4,4.0,0.0,0.0,2020-02-01 18:12:00
...,...,...,...,...
550,1434967.0,1404107.0,26599.0,2021-08-01 04:21:50
551,1435347.0,1404244.0,26601.0,2021-08-02 04:21:36
552,1435683.0,1404327.0,26602.0,2021-08-03 04:21:39
553,1437713.0,1405529.0,26608.0,2021-08-04 04:21:25


In [22]:
canada = canada.values[:, 0:3].astype(float)

In [23]:
min_max_scaler = MinMaxScaler()
min_max_scaler.fit(canada[:TRAIN_IDX])

MinMaxScaler()

In [24]:
canada_train = min_max_scaler.transform(canada[:TRAIN_IDX])
canada_val = min_max_scaler.transform(canada[TRAIN_IDX:VAL_IDX])
canada_test = min_max_scaler.transform(canada[VAL_IDX:])

In [10]:
# Learning Rate
lr = 0.01

# number of historic data points for fitting
input_steps = 1

# forecasting horizon
output_steps = 3

# number of epochs for training
num_epochs = 20000

In [28]:
training_data = covid_dataset.CovidDataset(canada_train, input_steps, output_steps)
train_dataloader = DataLoader(training_data, batch_size=1, shuffle=False)

val_data = covid_dataset.CovidDataset(canada_val, input_steps, output_steps)
val_dataloader = DataLoader(val_data, batch_size=1, shuffle=False)

test_data = covid_dataset.CovidDataset(canada_test, input_steps, output_steps)
test_dataloader = DataLoader(test_data, batch_size=1, shuffle=False)

In [45]:
model = sidartha_ode.SIDARTHEOde(len_data = output_steps).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 1000, gamma=0.9)
loss_fun = torch.nn.MSELoss()
min_loss = 1

##################################################################
for data, y_exact in tqdm(train_dataloader):
    optimizer.zero_grad()    
    y_approx = model(I0=data[0,0,0], H0=data[0,0,1], E0=data[0,0,2])
    loss = loss_fun(y_approx[1:, 
                             [1, 6, 7]], # I, H, E
                    y_exact.squeeze())
    loss.backward()
    optimizer.step()
    scheduler.step()
    for param in optimizer.param_groups[0]['params']:
        if param.requires_grad:
            print(param.grad)

with torch.no_grad():
    for data, y_exact in tqdm(train_dataloader):
        y_approx = model(I0=data[0,0,0], H0=data[0,0,1], E0=data[0,0,2])
        loss = loss_fun(y_approx[1:, 
                             [1, 6, 7]], # I, H, E
                    y_exact.squeeze())
        print(loss)

  0%|          | 0/69 [00:00<?, ?it/s]

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


  0%|          | 0/69 [00:00<?, ?it/s]

tensor(0.5740)
tensor(0.5740)
tensor(0.5740)
tensor(0.5740)
tensor(0.5740)
tensor(0.5740)
tensor(0.5740)
tensor(0.5740)
tensor(0.2542)
tensor(0.2542)
tensor(0.2542)
tensor(0.2542)
tensor(0.2542)
tensor(0.2542)
tensor(0.2542)
tensor(0.2531)
tensor(0.2531)
tensor(0.2531)
tensor(0.2531)
tensor(0.2531)
tensor(0.2531)
tensor(0.2531)
tensor(0.2531)
tensor(0.2531)
tensor(0.2531)
tensor(0.2531)
tensor(0.2531)
tensor(0.2531)
tensor(0.2531)
tensor(0.2531)
tensor(0.2531)
tensor(0.2531)
tensor(0.2532)
tensor(0.2532)
tensor(0.2533)
tensor(0.2533)
tensor(0.2534)
tensor(0.2534)
tensor(0.2535)
tensor(0.2534)
tensor(0.2534)
tensor(0.2532)
tensor(0.2533)
tensor(0.2539)
tensor(0.2544)
tensor(0.2553)
tensor(0.2557)
tensor(0.2564)
tensor(0.2577)
tensor(0.2589)
tensor(0.2609)
tensor(0.2630)
tensor(0.2660)
tensor(0.2709)
tensor(0.2754)
tensor(0.2820)
tensor(0.2895)
tensor(0.2954)
tensor(0.3067)
tensor(0.3171)
tensor(0.3286)
tensor(0.3408)
tensor(0.3579)
tensor(0.3770)
tensor(0.3986)
tensor(0.4311)
tensor(0.4

In [44]:
for param in optimizer.param_groups[0]['params']:
    if param.requires_grad:
        print(param.grad)

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [ ]:
y_approx

In [32]:
model.gamma

Parameter containing:
tensor([0.4560], requires_grad=True)

In [ ]:

y_exact